# Парсер характеристик 2 линеек: одной - успешной, другой - провальной
Цель тетради: собрать информацию по 2-м линейкам смартфонов:
1. Redmi - популярная в России и за рубежом линейка смартфонов от крупного китайского производителя Xiaomi. Начал рост популярности с 2016 года. Сейчас имеет значительный успех на рынках различных стран.
2. Nokia - популярный бренд смартфонов в 90-х, 2000-х. С 2016 года популярность Nokia начала значительно сокращаться. В настоящий момент предпринимались попытки реанимировать бренд, но они не увенчались успехом.

В резульате должны получиться 2 дадафрейма:
* Redmi - Датафрейм с характеристиками смартфоном Xiaomi Redmi
* Nokia - Датафрейм с характеристиками смартфонов бренда Nokia

Данные надо откуда-то взять.

За парсинг датасета из любого источника на ваш вкус вы получаете 20 баллов
За сбор датасета из разных источников (несколько готовых датасетов, но собранных из разных источников и объединенные общей логикой) вы получаете 10 баллов
За просто готовый датасет – 5 баллов
Если вы взяли готовый датасет и обогатили его, спарсив что-то, вы получаете 20 баллов. Тут важно спарсить что-то полезное, обогащающее датасет. За парсинг book to scrape ради галочки, вы баллов не получите. Не забудьте вставить ссылки на всё, что используете в итоговый блокнот и подробно описать свои действия.

Рекомендация к парсингу: Данное условие не является строгим и обязательным, а скорее рекомендация для вас. Если вы решили парсить данные, рекомендуем убедиться, что вы сможете собрать хотя бы 4-5 числовых признаков и несколько категориальных для вашей модели (если вы собираетесь что-то предсказывать). При этом также рекомендуем собрать хотя бы 1000 наблюдений для модели.

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

___

# Nokia

## Парсим первый датафрейм - Redmi Sales

In [3]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Nokia&dayRange=730&endDate=1686678585480&startDate=1623606585480&categoryId=9355&offset=0&limit=50&sorting=-datelastsold&tabName=SOLD&tz=Europe%2FMoscow")

#Ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_8404\3108897971.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_8404\3108897971.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

Парсим названия и числовые характеристики (Цены, продажи)

In [4]:
def table_parser(soup: BeautifulSoup):
    page_data = pd.DataFrame()
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        last_sale = product.find('td', class_='research-table-row__item research-table-row__dateLastSold').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link, "Last_Sale": last_sale}
        page_data = page_data.append(data, ignore_index=True)
    return page_data

In [6]:
df = pd.DataFrame()
for _ in tqdm(range(100)): #Решено спарсить 100 страниц
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    data = table_parser(soup)
    df = pd.concat([df, data], ignore_index=True)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

100%|██████████| 100/100 [20:26<00:00, 12.26s/it]


In [7]:
df.head()

,Title,Price,Shipping_cost,Sales_Count,Sales_Value,Link,Last_Sale
0,Nokia 3.1 C (TA1141) 32GB (Cricket) Smartphone...,$31.59,$5.00,34,"$1,074.14",https://www.ebay.com/itm/275016352540?nordt=tr...,"Jun 13, 2023"
1,"Nokia 6 Dual SIM 5.5"" 4GB RAM 16MP Wi-Fi GPS B...",$103.50,$0.00,4,$414.00,https://www.ebay.com/itm/203145222261?nordt=tr...,"Jun 13, 2023"
2,NEW Nokia 8 V 5G UW TA1257 Verizon Unlocked LT...,$161.24,$0.00,8,"$1,289.92",https://www.ebay.com/itm/295688872962?nordt=tr...,"Jun 13, 2023"
3,Original Nokia N8 N8-00 3G WIFI GPS 12MP Touch...,$42.95,$5.50,26,"$1,116.79",https://www.ebay.com/itm/164928279178?nordt=tr...,"Jun 13, 2023"
4,Nokia G50 5G | Android 11| Unlocked Smartphone...,$87.00,$8.22,1,$87.00,https://www.ebay.com/itm/145123725321?nordt=tr...,"Jun 13, 2023"


In [9]:
df.to_csv("Nokia Sales.tsv", sep='\t', index=False)

___

## Парсим характеристики моделей

In [10]:
df=pd.read_csv("Nokia Sales.tsv", delimiter = "\t")

In [11]:
Links = df.Link.tolist()
Links[0:10]

['https://www.ebay.com/itm/275016352540?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/203145222261?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/295688872962?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/164928279178?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/145123725321?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/166168230447?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/403640590392?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/404078507679?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/204363476374?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/225615294208?nordt=true&orig_cvip=true&rt=nc']

In [12]:
while np.nan in Links:
    Links.remove(np.nan)
None in Links

False

In [13]:
len(Links)

4747

In [14]:
import time
import random

characteristics_df = pd.DataFrame()
undone = []

# Задержка перед каждым запросом
def delay():
    time.sleep(random.uniform(1, 5))  # Измените значения задержки по своему усмотрению

def req_parser(url):
    global characteristics_df, undone

    try:
        # Отправляем GET-запрос и получаем ответ
        response = requests.get(url)

        # Получаем HTML-код страницы
        html = response.text

        # Создаем объект BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Находим родительский элемент
        parent_element = soup.find('div', class_='ux-layout-section--features')

        # Initialize an empty dictionary to store the characteristics
        characteristics = {"Link": url}
        done = []

        # Find all the characteristic rows
        rows = parent_element.find_all('div', class_='ux-layout-section-evo__row')

        # Iterate over each row and extract the characteristic name and value
        for row in rows:
            name_element = row.find_all('div', class_='ux-labels-values__labels')
            value_element = row.find_all('div', class_='ux-labels-values__values-content')

            for name, value in zip(name_element, value_element):
                if name and value:
                    name_text = name.get_text(strip=True)
                    value_text = value.get_text(strip=True)
                    if name_text not in done and value_text not in done:
                        characteristics[name_text] = value_text
                        done.append(name_text)
                        done.append(value_text)

        characteristics_df = characteristics_df.append(characteristics, ignore_index=True)

        # Задержка перед следующим запросом
        delay()

    except:
        print("Нащальника, не получилось:", url)
        undone.append(url)


for url in tqdm(Links):
    req_parser(url)

characteristics_df.to_csv("Nokia Characteristics.tsv", sep='\t', index=False)

 44%|████▍     | 2085/4747 [2:48:30<2:48:12,  3.79s/it]

Нащальника, не получилось: https://www.ebay.com/itm/225557179140?nordt=true&orig_cvip=true&rt=nc


 48%|████▊     | 2255/4747 [3:02:32<4:08:46,  5.99s/it]

Нащальника, не получилось: https://www.ebay.com/itm/373866317508?nordt=true&orig_cvip=true&rt=nc


 59%|█████▉    | 2815/4747 [3:46:50<2:03:24,  3.83s/it]

...

 95%|█████████▍| 4494/4747 [6:00:33<11:57,  2.83s/it]

Нащальника, не получилось: https://www.ebay.com/itm/354650172585?nordt=true&orig_cvip=true&rt=nc


 97%|█████████▋| 4606/4747 [6:09:14<16:32,  7.04s/it]

Нащальника, не получилось: https://www.ebay.com/itm/115732087182?nordt=true&orig_cvip=true&rt=nc


100%|██████████| 4747/4747 [6:20:39<00:00,  4.81s/it]


In [16]:
characteristics_df.to_csv("Nokia Characteristics.tsv", sep='\t', index=False)